In [4]:
import torch
import torch.nn as nn
import numpy as np

In [70]:
class MLPCritic(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_sizes=[256, 256]):
        super(MLPCritic, self).__init__()

        self.obs_dim = obs_dim
        self.act_dim = act_dim
        self.layer_sizes = [obs_dim + act_dim] + hidden_sizes + [1]

        self.layers = nn.ModuleList()
        # Hidden layers
        for h_i in range(len(self.layer_sizes) - 2):
            self.layers += [nn.Linear(self.layer_sizes[h_i], self.layer_sizes[h_i + 1]),
                            nn.Sigmoid()]
        # Output layer
        self.layers += [nn.Linear(self.layer_sizes[-2], self.layer_sizes[-1]),
                        nn.Identity()]

    def forward(self, obs, act):
        x = torch.cat([obs, act], dim=-1)
        hid_activation = []
        # Hidden layers
        for h_i in range(len(self.layers) - 2):
            print(self.layers[h_i])
            x = self.layers[h_i](x)
            # Store activation
            if h_i % 2 == 1:
                hid_activation.append(x)
        # Output layer
        x = self.layers[-2](x)
        x = self.layers[-1](x)
        return torch.squeeze(x, -1), hid_activation


class MLPActor(nn.Module):
    def __init__(self, obs_dim, act_dim, act_limit, hidden_sizes=[256, 256]):
        super(MLPActor, self).__init__()
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        self.act_limit = act_limit
        self.layer_sizes = [obs_dim] + hidden_sizes + [act_dim]

        self.layers = nn.ModuleList()
        # Hidden layers
        for h_i in range(len(self.layer_sizes) - 2):
            self.layers += [nn.Linear(self.layer_sizes[h_i], self.layer_sizes[h_i + 1]),
                            nn.Sigmoid()]
        # Output layer
        self.layers += [nn.Linear(self.layer_sizes[-2], self.layer_sizes[-1]),
                        nn.Tanh()]

    def forward(self, obs):
        x = obs
        hid_activation = []
        # Hidden layers
        for h_i in range(len(self.layers) - 2):
            
            x = self.layers[h_i](x)
            # Store activation
            if h_i % 2 == 1:
                hid_activation.append(x)
        # Output layer
        x = self.layers[-2](x)
        x = self.layers[-1](x)
        return self.act_limit * x, hid_activation


class MLPActorCritic(nn.Module):
    def __init__(self, obs_dim, act_dim, act_limit, critic_hidden_sizes=[256, 256], actor_hidden_sizes=[256, 256]):
        super(MLPActorCritic, self).__init__()
        self.q1 = MLPCritic(obs_dim, act_dim, critic_hidden_sizes)
        self.q2 = MLPCritic(obs_dim, act_dim, critic_hidden_sizes)
        self.pi = MLPActor(obs_dim, act_dim, act_limit, actor_hidden_sizes)

    def act(self, obs):
        with torch.no_grad():
            a, _ = self.pi(obs)
            return a.cpu().numpy()

In [71]:
obs_dim = 15
act_dim = 5
act_limit = 1
critic = MLPCritic(obs_dim, act_dim)
actor = MLPActor(obs_dim, act_dim, act_limit)

In [72]:
m = 100
obs = torch.as_tensor(np.random.rand(m, obs_dim), dtype=torch.float32)
act = torch.as_tensor(np.random.rand(m, act_dim), dtype=torch.float32)
q, q_hid_activation = critic(obs, act)
a, a_hid_activation = actor(obs)

Linear(in_features=20, out_features=256, bias=True)
Sigmoid()
Linear(in_features=256, out_features=256, bias=True)
Sigmoid()


In [75]:
critic.parameters()
for p in critic.parameters():
    if p.requires_grad:
         print(p.name, p.data)

None tensor([[ 0.0531, -0.1832,  0.0334,  ..., -0.2100, -0.1570, -0.0378],
        [-0.0560, -0.1791, -0.1595,  ...,  0.1052,  0.0930,  0.0297],
        [ 0.0045, -0.0834, -0.1346,  ...,  0.1215, -0.0272, -0.1203],
        ...,
        [ 0.1842, -0.1546, -0.0197,  ..., -0.0616,  0.0538, -0.0298],
        [-0.1351, -0.1441,  0.2059,  ..., -0.1177, -0.0467,  0.1636],
        [-0.0337,  0.1200,  0.0375,  ..., -0.1362, -0.1893,  0.0973]])
None tensor([ 0.0584,  0.1826,  0.1727,  0.0309,  0.0184, -0.0012,  0.2171,  0.0029,
        -0.1473,  0.1677,  0.0464,  0.1544,  0.1863,  0.1003, -0.2109,  0.1163,
         0.1077, -0.1256, -0.0921, -0.0257, -0.2108, -0.1411,  0.2233, -0.0551,
        -0.2153, -0.0741,  0.0124, -0.1674,  0.0214, -0.1084,  0.0072,  0.0530,
        -0.1234, -0.0430,  0.1154, -0.1921,  0.0752,  0.1646, -0.2199, -0.0754,
        -0.0467, -0.1953,  0.0834,  0.0946, -0.1838,  0.0632,  0.0740, -0.1502,
        -0.1853,  0.1969, -0.0155, -0.1755, -0.1736, -0.0732,  0.1368,  0.03

In [55]:
print(critic)
print(actor)

MLPCritic(
  (layers): ModuleList(
    (0): Linear(in_features=20, out_features=256, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Identity()
  )
)
MLPActor(
  (layers): ModuleList(
    (0): Linear(in_features=15, out_features=256, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=256, out_features=5, bias=True)
    (5): Tanh()
  )
)


In [56]:
critic.layers

ModuleList(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=256, out_features=1, bias=True)
  (5): Identity()
)

In [27]:
hid_a = hid_activation[0]
hid_a.mean(axis=0)

tensor([0.6067, 0.5047, 0.4744, 0.4898, 0.5349, 0.5409, 0.4726, 0.5370, 0.4718,
        0.3470, 0.3877, 0.5362, 0.3976, 0.5017, 0.3958, 0.5525, 0.5401, 0.3854,
        0.5290, 0.5130, 0.4945, 0.4786, 0.5506, 0.4881, 0.5183, 0.3692, 0.6020,
        0.3939, 0.5712, 0.5318, 0.6325, 0.5938, 0.4832, 0.5813, 0.3648, 0.4099,
        0.5243, 0.5920, 0.5059, 0.5398, 0.3487, 0.5012, 0.5444, 0.4800, 0.6469,
        0.4534, 0.4393, 0.5691, 0.4726, 0.5828, 0.4657, 0.5383, 0.4352, 0.3629,
        0.5125, 0.6026, 0.4704, 0.5292, 0.4727, 0.4528, 0.4770, 0.5008, 0.4778,
        0.5587, 0.5814, 0.5431, 0.4608, 0.6139, 0.6130, 0.4333, 0.4183, 0.3449,
        0.6047, 0.3676, 0.5674, 0.5013, 0.5519, 0.5632, 0.4392, 0.3591, 0.4591,
        0.5530, 0.6337, 0.3952, 0.4739, 0.4734, 0.4242, 0.4878, 0.5078, 0.6197,
        0.4293, 0.6240, 0.4365, 0.4395, 0.5399, 0.5173, 0.6532, 0.5311, 0.3310,
        0.5044, 0.5077, 0.3787, 0.4826, 0.5108, 0.5040, 0.4619, 0.4056, 0.5470,
        0.5123, 0.4193, 0.4274, 0.5585, 

In [77]:
kl_loss = nn.KLDivLoss(reduction='sum')
kl_loss(torch.cat(hid_activation, dim=1).mean(axis=0), rho)

tensor(-89.4653, grad_fn=<KlDivBackward>)

In [79]:
rho = torch.as_tensor(0.05, dtype=torch.float32)
beta = 0.5
sparsity_penalty = torch.nn.functional.kl_div(torch.cat(hid_activation, dim=1).mean(axis=0), rho, reduction='sum')
sparsity_penalty

tensor(-89.4653, grad_fn=<KlDivBackward>)

In [80]:
sparsity_penalty = torch.nn.functional.kl_div(rho, torch.cat(hid_activation, dim=1).mean(axis=0), reduction='sum')
sparsity_penalty

tensor(-187.5761, grad_fn=<KlDivBackward>)

In [82]:
torch.nn.functional.kl_div(torch.as_tensor(0.5, dtype=torch.float32), torch.as_tensor(0.5, dtype=torch.float32))

tensor(-0.5966)

In [84]:

rho_hat = torch.as_tensor(0.5, dtype=torch.float32)
rho = torch.ones(rho_hat.shape)
torch.sum(rho * torch.log(rho/rho_hat) + (1 - rho) * torch.log((1 - rho)/(1 - rho_hat)))

tensor(nan)

In [85]:
rho_hat.shape

torch.Size([])

In [92]:
rho_hat = torch.cat(hid_activation, dim=1).mean(axis=0)
rho_hat.shape
rho = torch.ones(rho_hat.shape)*0.05
torch.sum(rho * torch.log(rho/rho_hat) + (1 - rho) * torch.log((1 - rho)/(1 - rho_hat)))

tensor(258.0015, grad_fn=<SumBackward0>)